In [145]:
import torch
from stable_baselines3 import PPO
import gymnasium as gym
import numpy as np
import imageio
import subprocess
import shutil
import time
import os

In [146]:
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        # Access relevant state and action variables
        
        #state
        z = obs[0]                  # Height of the robot
        a = obs[1]                  # Angle of the robot
        a_hip = obs[2]              # Angle of the hip
        a_knee = obs[3]             # Angle of the knee
        a_ankle = obs[4]            # Angle of the ankle
        v_x = obs[5]                # Velocity in x direction
        v_z = obs[6]                # Velocity in z direction
        a_d = obs[7]                # Angular velocity 
        a_hip_d = obs[8]            # Angular velocity of the hip
        a_knee_d = obs[9]           # Angular velocity of the knee
        a_ankle_d = obs[10]         # Angular velocity of the ankle
        
        #action
        torque_hip = action[0]      # Torque applied to the hip
        torque_knee = action[1]     # Torque applied to the knee
        torque_ankle = action[2]    # Torque applied to the ankle

        #different criteria for reward
        energy_used = np.sum(np.square(action))  # Simplistic energy calculation

        # Custom reward logic
        custom_reward = z * np.exp(-a**2/(np.pi/2)) * 4             # Reward for height
        custom_reward += a_d * np.exp(-(a-np.pi)**2/4*np.pi) * 1      # Reward for angular velocity
        custom_reward += 3*np.exp(-(a-(2*np.pi))**2/(np.pi))      # Reward for 2pi angle
        #custom_reward -= energy_used * 0.01                        # Penalize energy consumption

        # used by openai
        # backroll = -obs[7]
        # height = obs[0]
        # vel_act = action[0] * obs[8] + action[1] * obs[9] + a[2] * obs[10]
        # backslide = -obs[5]
        # reward = backroll * (1.0 + .3 * height + .1 * vel_act + .05 * backslide)

        if done:
            custom_reward -= 10  # Heavy penalty for falling

        return obs, custom_reward, done, truncated, info

In [147]:
healthy_reward = 0.8
healthy_z_range = (0.2, float("inf"))
healthy_angle_range = (-np.pi/4, 2*np.pi+np.pi/4)
reset_noise_scale = 5e-3
exclude_current_positions_from_observation = True

env = gym.make('Hopper-v4', render_mode='rgb_array', healthy_reward=healthy_reward, healthy_z_range=healthy_z_range, healthy_angle_range=healthy_angle_range, reset_noise_scale=reset_noise_scale, exclude_current_positions_from_observation=exclude_current_positions_from_observation)
env = CustomRewardWrapper(env)

model = PPO("MlpPolicy", env, verbose=1)

# Train the model
n_learning_steps = 30000
model.learn(total_timesteps=n_learning_steps)

# Save the model
model.save("hopper")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 42.4     |
|    ep_rew_mean     | 176      |
| time/              |          |
|    fps             | 1099     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 55.5        |
|    ep_rew_mean          | 219         |
| time/                   |             |
|    fps                  | 713         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009972944 |
|    clip_fraction        | 0.139       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
vec_env = model.get_env()
obs = vec_env.reset()

writer = imageio.get_writer('hopper-flip.mp4', fps=100)

N_step = 100000
for i in range(N_step):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    print(obs[0][1])
    writer.append_data(vec_env.render("rgb_array"))
    #VecEnv resets automatically
    if done:
      print("Episode finished after {} timesteps".format(i+1))
      break
      obs = vec_env.reset()

writer.close()

# d = time.strftime("%Y-%m-%d-%H-%M-%S")
# current_py_file = os.getcwd() + '/hopper.ipynb'
# new_py_file = os.getcwd() + '/Run/hopper%s.ipynb'%(d)

# shutil.copyfile(current_py_file, new_py_file)
# shutil.copyfile('hopper-flip.mp4', os.getcwd()+'/Render/hopper-flip%s.mp4'%(d))

-0.0013031148904816582
0.0029772938101520047
0.004916829970671255
0.0055312834955815755
0.0055455907065443885
0.0053400815685565655
0.005104965268702393
0.004927446563248502
0.004840414945353307
0.004849359479599823
0.005040367255142958
0.005925570408564885
0.007305031395670563
0.008930308794081274
0.010639887334770465
0.012333695335797245
0.013839542027135021
0.01486862279496708
0.015506973208555022
0.015839925032945863
0.015941746157992147
0.015894107392359583
0.015774611592855585
0.015651896903613724
0.015572541411073223
0.015566099125063449
0.015653758086180747
0.015847414375618183
0.016150125095703804
0.016557348529912957
0.017058546562919236
0.017638919786985365
0.018281133609926087
0.018966929166851405
0.019678530433541303
0.020399777713702084
0.021116944312237318
0.021819225688165182
0.02249892211744238
0.023151360102522802
0.023774611518656116
0.024369073768367146
0.02493697085567598
0.02548182780453945
0.026007960962803684
0.026520016289051453
0.02702257763800171
0.0275198575

In [ ]:
np.pi/2

1.5707963267948966